In [58]:
from datetime import datetime, timedelta
import pytz
import pandas as pd
from tqdm import tqdm
from modeler.modeler import Modeler as m
from processor.processor import Processor as p
from database.market import Market
from database.adatabase import ADatabase
import numpy as np
import matplotlib.pyplot as plt
import pickle
import matplotlib.dates as mdates
import statistics

In [59]:
market = Market()
speculation_db = ADatabase("bitcoin_speculation")
training_year = 4
signal = 0.1
req = 0.1
positions = 1
today = datetime(2022,8,19)
year = today.year
week =  today.isocalendar()[1]
# today = datetime.now()

In [60]:
for date in daterange(dates):
    p

33

In [61]:
market.connect()
sp5 = market.retrieve("sp500")
market.disconnect()

In [62]:
speculation_db.connect()
recs = speculation_db.retrieve('recs')
speculation_db.disconnect()

In [63]:
relevant_recs = recs[(recs["date"]==today) & (recs["training_year"]==training_year)]

In [64]:
relevant_recs["prediction"] = (relevant_recs["xgb_prediction"] + relevant_recs["skl_prediction"] + relevant_recs["cat_prediction"]) / 3

In [65]:
relevant_recs

,date,ticker,training_year,skl_prediction,skl_score,xgb_prediction,xgb_score,cat_prediction,cat_score,prediction
228,2022-08-19,BTC,4,22943.010679,0.99374,27638.732422,0.991037,27647.992334,0.992605,26076.578478
230,2022-08-19,BTC,4,22943.010679,0.99374,27638.732422,0.991037,27647.992334,0.992605,26076.578478


In [66]:
market.connect()
ticker_prices = []
prices = market.retrieve("pdr_crypto")
prices = p.column_date_processing(prices)
prices["year"] = [x.year for x in prices["date"]]
prices["week"] = [x.week for x in prices["date"]]
relevant_data = prices[prices["date"]==today]
ticker_prices.append(relevant_data)
market.disconnect()

In [67]:
tp = pd.concat(ticker_prices)

In [68]:
final = tp.merge(relevant_recs,on=["date","ticker"])
final["delta"] = (final["prediction"] - final["adjclose"]) / final["adjclose"]

In [69]:
todays_rec = final[["date","adjclose","prediction","delta"]][final["delta"]>=signal].sort_values("delta",ascending=False).head(positions)

In [70]:
relevant_recs

,date,ticker,training_year,skl_prediction,skl_score,xgb_prediction,xgb_score,cat_prediction,cat_score,prediction
228,2022-08-19,BTC,4,22943.010679,0.99374,27638.732422,0.991037,27647.992334,0.992605,26076.578478
230,2022-08-19,BTC,4,22943.010679,0.99374,27638.732422,0.991037,27647.992334,0.992605,26076.578478


In [71]:
speculation_db.connect()
portfolios = speculation_db.retrieve("portfolios")
speculation_db.disconnect()

In [72]:
if portfolios.index.size < 1:
    positions_dictionary = {}
    portfolio = {"date":today}
    for position in range(positions):
        positions_dictionary[position] = {"cash":100000/positions,"asset":{},"queue":{}}
    portfolio["positions"] = positions_dictionary
else:
    portfolios["positions"] = [pickle.loads(x) for x in portfolios["positions"]]
    portfolio = portfolios.sort_values("date",ascending=False).head(positions).to_dict("records")[0]

In [73]:
existing_assets = [portfolio["positions"][position]["asset"]["ticker"] for position in portfolio["positions"].keys() if "ticker" in portfolio["positions"][position]["asset"].keys()]

In [74]:
final

,date,high,low,open,close,volume,adjclose,ticker,year,quarter,week,training_year,skl_prediction,skl_score,xgb_prediction,xgb_score,cat_prediction,cat_score,prediction,delta
0,2022-08-19,23180.912109,22760.619141,23180.912109,22821.220703,2.595380e+10,22821.220703,BTC,2022,3,33,4,22943.010679,0.99374,27638.732422,0.991037,27647.992334,0.992605,26076.578478,0.142646
1,2022-08-19,23180.912109,22760.619141,23180.912109,22821.220703,2.595380e+10,22821.220703,BTC,2022,3,33,4,22943.010679,0.99374,27638.732422,0.991037,27647.992334,0.992605,26076.578478,0.142646


In [77]:
## looking for exits
speculation_db.connect()
for position in range(positions):
    position_dictionary = portfolio["positions"][position]
    if len(list(position_dictionary["asset"].keys())) > 0:
        asset = position_dictionary["asset"]
        ticker = asset["ticker"]
        amount = asset["amount"]
        price = asset["price"]
        exit_price = price * (1+req)
        spread = final[final["ticker"]==ticker].iloc[0]
        if spread["high"].item() >= exit_price:
            position_dictionary["cash"] = position_dictionary["cash"] + (exit_price * amount)
            trade = asset
            trade["sell_date"] = today
            trade["sell_price"] = exit_price
            trade["position"] = position
            trade["req"] = req
            trade["training_year"] = training_year
            trade["signal"] = signal
            speculation_db.store("ledger",pd.DataFrame([trade]))
            print("sell",exit_price,spread["high"].item())
            position_dictionary["asset"] = {}
        else:
            if int((today-asset["date"]).days) >= 2:
                print("due date")
                position_dictionary["cash"] = position_dictionary["cash"] + (spread["adjclose"].item() * amount)
                trade = asset
                trade["sell_date"] = today
                trade["sell_price"] = spread["adjclose"].item()
                trade["position"] = position
                trade["req"] = req
                trade["training_year"] = training_year
                trade["signal"] = signal
                speculation_db.store("ledger",pd.DataFrame([trade]))
                position_dictionary["asset"] = {}
            else:
                print("hold")
    else:
        print(position_dictionary["cash"])
speculation_db.disconnect()

102717.859375


In [79]:
## looking for entries
for position in range(positions):
    position_dictionary = portfolio["positions"][position]
    if len(list(position_dictionary["queue"].keys())) > 0:
        queue = position_dictionary["queue"]
        ticker = queue["ticker"]
        amount = queue["amount"]
        price = queue["price"]
        spread = final[final["ticker"]==ticker].iloc[0]
        if (price >= spread["low"].item()) and (price <= spread["high"].item()):
            print("buy")
            position_dictionary["asset"] = {"date":today,"ticker":ticker,"amount":amount,"price":price}
            position_dictionary["cash"] = position_dictionary["cash"] - amount * price
            position_dictionary["queue"] = {}
        else:
            position_dictionary["queue"] = {}
            print("fail")
    else:
        print(position_dictionary["cash"])

fail


In [80]:
portfolio

{'date': Timestamp('2022-08-17 00:00:00'),
 'positions': {0: {'cash': 102717.859375, 'asset': {}, 'queue': {}}}}

In [81]:
## looking for recs
for position in range(positions):
    position_dictionary = portfolio["positions"][position]
    ## recommendation impossible
    if (len(list(position_dictionary["queue"].keys())) > 0) or (len(list(position_dictionary["asset"].keys())) > 0) or (position_dictionary["cash"]<=1.0) or (position >= todays_rec.index.size):
        print(position_dictionary["queue"])
    else:
        cash = position_dictionary["cash"]
        position_rec = todays_rec.iloc[position]
        price = position_rec["adjclose"]
        ticker = "BTC"
        amount = int(cash/price)
        if amount < 1 or ticker in existing_assets:
            continue
        else:
            position_dictionary["queue"] = {"ticker":ticker,"price":price,"amount":amount}
## storing

In [82]:
portfolio["date"] = today

In [83]:
portfolio["positions"] = pickle.dumps(portfolio["positions"])

In [84]:
portfolio

{'date': datetime.datetime(2022, 8, 19, 0, 0),
 'positions': b'\x80\x04\x95\xc4\x00\x00\x00\x00\x00\x00\x00}\x94K\x00}\x94(\x8c\x04cash\x94\x8c\x15numpy.core.multiarray\x94\x8c\x06scalar\x94\x93\x94\x8c\x05numpy\x94\x8c\x05dtype\x94\x93\x94\x8c\x02f8\x94\x89\x88\x87\x94R\x94(K\x03\x8c\x01<\x94NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00t\x94bC\x08\x00\x00\x00\xc0\xdd\x13\xf9@\x94\x86\x94R\x94\x8c\x05asset\x94}\x94\x8c\x05queue\x94}\x94(\x8c\x06ticker\x94\x8c\x03BTC\x94\x8c\x05price\x94h\x05h\x0bC\x08\x00\x00\x00 NI\xd6@\x94\x86\x94R\x94\x8c\x06amount\x94K\x04uus.'}

In [85]:
speculation_db.connect()
speculation_db.store("portfolios",pd.DataFrame([portfolio]))
speculation_db.disconnect()